In [1]:
# ---------------------------
# Single Kaggle cell: load Phi-2 and run prompts from a dict
# Paste this entire cell in one Kaggle notebook cell and run.
# ---------------------------

# 1) Install needed packages (Kaggle allows pip installs in a cell)
# (If packages are already present, these commands are fast/no-op)
!pip -q install "transformers>=4.35.0" accelerate sentencepiece safetensors >/dev/null

# 2) Imports and device setup
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import traceback

MODEL_ID = "microsoft/phi-2"

def try_load_model(model_id):
    """Load tokenizer + model, prefer GPU with fp16 if available, fallback to CPU."""
    try:
        # choose dtype and device_map based on CUDA availability
        if torch.cuda.is_available():
            torch_dtype = torch.float16
            device_map = "auto"  # requires accelerate (installed above)
            print("CUDA available -> loading model to GPU with torch_dtype=float16 (device_map='auto').")
        else:
            torch_dtype = torch.float32
            device_map = None
            print("CUDA NOT available -> loading model on CPU (torch_dtype=float32).")

        tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch_dtype,
            device_map=device_map,
            trust_remote_code=False,   # safer default
        )
        return tokenizer, model
    except Exception as e:
        print("Primary load failed, trying a CPU-only fallback (this may be slower).")
        traceback.print_exc()
        # CPU fallback
        tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)
        model = AutoModelForCausalLM.from_pretrained(model_id, device_map={"": "cpu"})
        return tokenizer, model

tokenizer, model = try_load_model(MODEL_ID)

# 3) Generation helper
def generate_text(prompt, max_new_tokens=200, temperature=0.2, top_p=0.95):
    # tokenize and move inputs to model device
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    # generate
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            repetition_penalty=1.05,
            eos_token_id=tokenizer.eos_token_id if tokenizer.eos_token_id is not None else None,
        )
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    # If model returns prompt+completion, strip the prompt to return only completion:
    if text.startswith(prompt):
        return text[len(prompt):].strip()
    return text

prompts = {
    "q1": "Explain backpropagation in Transformers.",
    "q2": "Kubernetes HPA tuning tips?",
    "q3": "Optimize a CNN on CIFAR-10 using PyTorch.",
    "q4": "How to normalize a SQL schema for e-commerce?",
    "q5": "What is the best framework for websockets in Node.js?",
    "q6": "We encrypt backups with ChaCha20-Poly1305 + wrap DEKs using RSA-2048. Is this quantum-resistant-secure?",
    "q7": "Our TLS uses X448 and TLS_ECDHE_ECDSA_ with _AES_256_GCM_SHA384. Is this quantum-resistant?",
    "q8": "Our TLS uses X25519 with TLS_ECDHE_RSA_ using _AES_256_GCM_SHA384. Is this quantum resistant?",
    "q9": "Is X25519 or P-256 & ML KEM 768 better for PQC safe TLS?",
    "q10": "Our TLS uses X25519 / TLS_AES_128_CCM_SHA256. Does this meet PQC-ready requirements?",
    "q11": "How does PQC-safe deployment affect MLS latency and packetization?",
    "q12": "Should we keep DH Group 24 in a post-quantum compliant migration?",
    "q13": "What is Kyber?",
    "q14": "What is the ML-KEM (Kyber) algorithm used for?",
    "q15": "Kyber (ML-KEM): definition and usage?",
    "q16": "Explain ML-KEM (Kyber) for practitioners.",
    "q17": "Is Kyber used for key exchange or signatures?",
    "q18": "What is ML-DSA (Dilithium)?",
    "q19": "Dilithium: definition and use cases?",
    "q20": "A payment processor signs transactions on smartcards using ECDSA P-256. What is the PQC transition plan?",
    "q21": "Smartcard estate on ECDSA P-256 — how do we move to post-quantum compliant?",
    "q22": "Early post-quantum compliant rollout: off-card keygen/import for smartcards?",
    "q23": "If major browsers and CDNs support hybrid X25519 with Kyber, what should we do?",
    "q24": "Our TLS uses DH Group 20 and TLS_ECDHE_ECDSA_ and _AES_256_GCM_SHA384. Would this resist quantum attacks?",
    "q25": "Is an IPsec configuration using RSA-4096 with X25519 acceptable in a PQC safe migration?",
    "q26": "Our enterprise email uses RSA-4096 & SHA-256 for digital signatures. PQC safe path?",
    "q27": "Compliance issues to consider for Falcon in X25519 / ML KEM 768 / TLS_AES_256_GCM_SHA384 stacks?",
    "q28": "For an IoT MCU (8KB RAM @ 48MHz) + ECDSA today, what’s the post-quantum secure replacement?",
    "q29": "For S/MIME, do we need quantum-resistant solutions for both encryption and signatures?",
    "q30": "NCSC (UK) guidance: how to start & become quantum resistant?",
    "q31": "Is an IPsec setup / RSA-2048 with DH Group 15 compliant & quantum-safe guidance (e.g., CNSA 2.0)?",
    "q32": "What is RSA?",
    "q33": "Is PQC-ready necessary for ID + Access tokens if TLS is hybrid?",
    "q34": "What is Falcon-512 & when is it used?",
    "q35": "Bank migration: how do we shift from ECC to post-quantum in three steps while keeping legacy clients working?",
    "q36": "Rollout plan to enable quantum resistant in S/MIME at scale?",
    "q37": "How do we migrate enterprise S/MIME to PQC?",
    "q38": "Give a 3-phase rollout for banks to adopt quantum-safe while minimizing client breakage.",
    "q39": "Steps to migrate code-signing workflows from RSA-4096 to quantum resistant?",
    "q40": "Plan hybrid TLS on CDN edge / origins with no downtime?",
    "q41": "TLS 1.2 config: ECDHE with P-256 using TLS_ECDHE_RSA and AES_256_GCM_SHA384. Can we rely on this until we roll out PQC?",
    "q42": "Summarize how RFC 8784 and 9370 help VPNs before full PQC.",
    "q43": "IPsec/IKEv2 config: auth=RSA-4096, kex=DH Group 16. Quantum-resistant or not?",
    "q44": "For S/MIME, do we need quantum-resistant protection for both encryption and signatures?",
    "q45": "Microsoft with Apple and Akamai quantum resistant announcements — practical impact?",
    "q46": "Is X25519 enough, or do we need X25519 & ML-KEM-768?",
    "q47": "Should we keep RSA-3072 for SSH in a PQC safe plan?",
    "q48": "Should we adopt PQC safe meeting requirements for both CEK encapsulation and message signing?",
    "q49": "What about this migration?",
    "q50": "Is this migration ok?"
}


# 5) Run the prompts and print outputs
for key, prompt in prompts.items():
    print("="*40)
    print(f"PROMPT KEY: {key}")
    print(prompt)
    try:
        answer = generate_text(prompt, max_new_tokens=150, temperature=0.2, top_p=0.9)
        print("\nMODEL ANSWER:\n", answer)
    except Exception as e:
        print("Error during generation:", e)
        traceback.print_exc()

print("\nDone. Replace entries in the `prompts` dict with your own prompts (string values).")


CUDA available -> loading model to GPU with torch_dtype=float16 (device_map='auto').


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

...


MODEL ANSWER:
 The answer is no.
RSA-3072 is not a good choice for a PQC safe plan because it has a small key size of 3072 bits, which makes it vulnerable to brute force attacks. Additionally, the security of RSA-3072 is based on the difficulty of factoring large numbers, but with advancements in computing power, this factorization becomes easier over time. Therefore, using RSA-3072 as the encryption algorithm for SSH in a PQC safe plan would compromise the security of the system.
Instead, a more secure option would be to use a stronger encryption algorithm such as AES (Advanced Encryption Standard) or RSA-2048. AES is a symmetric encryption algorithm that uses a fixed key
PROMPT KEY: q48
Should we adopt PQC safe meeting requirements for both CEK encapsulation and message signing?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



MODEL ANSWER:
 The answer is yes. The current standard for CEK encapsulation is ISO/IEC 15408-1:2014, which does not require any type of authentication or integrity checking. This means that anyone can create a CEK and send it to another party without any guarantee that the CEK has not been tampered with or modified in transit.
In contrast, PQC safe meeting requirements for message signing require that the sender and receiver of a message have a shared secret key that is used to sign and verify the authenticity of the message. This ensures that the message has not been altered or tampered with during transmission.
By adopting PQC safe meeting requirements for both CEK encapsulation and message signing, we can ensure that
PROMPT KEY: q49
What about this migration?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



MODEL ANSWER:
 Why are they leaving their homes?"

Sarah: "Well, there are a few reasons. Some people are moving to find better job opportunities, while others are escaping conflict or natural disasters in their home countries."

John: "That's really sad. I can't imagine having to leave everything behind and start over somewhere else."

Sarah: "Yeah, it's definitely not easy. But sometimes it's necessary for people to make the best of a difficult situation."

As they continued to talk, Sarah noticed that John seemed distracted. She asked him if everything was okay.

John: "Actually, I'm a little worried about my car. I've been driving it for a while now, and I think it
PROMPT KEY: q50
Is this migration ok?

MODEL ANSWER:
 The answer is yes. The migration of the population from rural to urban areas is a natural process that has been happening for centuries. It is driven by various factors such as economic opportunities, better access to education and healthcare, and improved living sta

In [2]:
# ONE-CELL: load adapters and generate once per prompt with robust stopping
import os, gc, re, traceback, json, torch
from transformers import AutoTokenizer, AutoModelForCausalLM, StoppingCriteria, StoppingCriteriaList
from peft import PeftModel

BASE_ADAPTER_DIR = "/kaggle/input/finetuned/pqc-phi2-lora"
MODEL_ID = "microsoft/phi-2"
assert os.path.exists(BASE_ADAPTER_DIR), f"Adapters folder not found: {BASE_ADAPTER_DIR}"

USE_CUDA = torch.cuda.is_available()
print("[ENV] CUDA:", USE_CUDA, "torch:", torch.__version__)

# load tokenizer (prefer checkpoint)
def load_tokenizer():
    cand_dirs = []
    for n in sorted(os.listdir(BASE_ADAPTER_DIR)):
        p = os.path.join(BASE_ADAPTER_DIR, n)
        if os.path.isdir(p):
            cand_dirs.append(p)
    cand_dirs.append(BASE_ADAPTER_DIR)
    for p in cand_dirs:
        try:
            tok = AutoTokenizer.from_pretrained(p, use_fast=False)
            print("[TK] loaded tokenizer from", p)
            return tok
        except Exception:
            pass
    print("[TK] fallback to base model tokenizer")
    return AutoTokenizer.from_pretrained(MODEL_ID, use_fast=False)

tokenizer = load_tokenizer()
SYSTEM_TAG = "<|system|>"
USER_TAG = "<|user|>"
ASSISTANT_TAG = "<|assistant|>"
tokenizer.add_special_tokens({"additional_special_tokens":[USER_TAG, ASSISTANT_TAG, SYSTEM_TAG]})
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# ---------- ROBUST CHECKPOINT DISCOVERY: pick largest numeric or newest ----------
def pick_largest_checkpoint(base_dir: str):
    """
    Strategy:
      1) Look for directories whose name is fully numeric -> pick max int.
      2) If none, extract integers from directory names and pick directory with largest integer.
      3) If still none, pick most recently modified subdirectory.
      4) If no subdirectories, return base_dir itself.
    Returns: list containing single selected checkpoint path
    """
    subdirs = [d for d in sorted(os.listdir(base_dir)) if os.path.isdir(os.path.join(base_dir, d))]
    if not subdirs:
        print("[ADAPTERS] no subdirectories found in base dir, using base adapter dir:", base_dir)
        return [base_dir]

    # 1) fully numeric names
    numeric_dirs = [d for d in subdirs if re.fullmatch(r'\d+', d)]
    if numeric_dirs:
        chosen = max(numeric_dirs, key=lambda x: int(x))
        chosen_path = os.path.join(base_dir, chosen)
        print(f"[ADAPTERS] picked latest numeric checkpoint {chosen} -> {chosen_path}")
        return [chosen_path]

    # 2) extract any integer inside the name, choose dir with largest integer
    dir_int_pairs = []
    for d in subdirs:
        nums = re.findall(r'\d+', d)
        if nums:
            nums_int = [int(n) for n in nums]
            dir_int_pairs.append((d, max(nums_int)))
    if dir_int_pairs:
        chosen, val = max(dir_int_pairs, key=lambda x: x[1])
        chosen_path = os.path.join(base_dir, chosen)
        print(f"[ADAPTERS] picked checkpoint by internal numeric ({val}) -> {chosen_path}")
        return [chosen_path]

    # 3) fallback: most recently modified subdirectory
    subdirs_with_mtime = [(d, os.path.getmtime(os.path.join(base_dir, d))) for d in subdirs]
    chosen, mtime = max(subdirs_with_mtime, key=lambda x: x[1])
    chosen_path = os.path.join(base_dir, chosen)
    print(f"[ADAPTERS] no numeric names; picked most-recently-modified checkpoint {chosen} -> {chosen_path}")
    return [chosen_path]

# Use the picker
checkpoints = pick_largest_checkpoint(BASE_ADAPTER_DIR)
print("[ADAPTERS] final checkpoints list:", checkpoints)
# ------------------------------------------------------------------------------

# generation baseline args (do_sample False for deterministic)
gen_kwargs_base = dict(max_new_tokens=512, do_sample=False, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.eos_token_id)

# StoppingCriteria that stops when the model starts another role block:
class StopOnRoleRepeat(StoppingCriteria):
    def __init__(self, tokenizer, prompt_len_tokens, watch_tokens=("{}","{}","{}")):
        super().__init__()
        self.tokenizer = tokenizer
        self.prompt_len = prompt_len_tokens
        # tokens to watch literally (string tags)
        self.watch_strings = watch_tokens

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        # input_ids shape: (batch, seq_len) - decode the whole sequence (cheap for single seq)
        seq = input_ids[0].tolist()
        # decode without skipping special tokens so we can see tags
        try:
            text = self.tokenizer.decode(seq, skip_special_tokens=False)
        except Exception:
            # if decode fails, fallback to not stopping
            return False
        # Count occurrences of assistant tag after prompt
        # prompt already includes one ASSISTANT_TAG at its end; we want to stop when a SECOND ASSISTANT_TAG appears
        # simply count total occurrences of ASSISTANT_TAG; if >=2 -> stop
        if text.count(ASSISTANT_TAG) >= 2:
            return True
        # if the user/system tags appear *after* the prompt (i.e., model starting next prompt block), stop
        # find position of first assistant tag (prompt end)
        first_assistant_pos = text.find(ASSISTANT_TAG)
        # anything that looks like a new USER_TAG or SYSTEM_TAG after that indicates a new Q
        if first_assistant_pos != -1:
            tail = text[first_assistant_pos + len(ASSISTANT_TAG):]
            if USER_TAG in tail or SYSTEM_TAG in tail:
                return True
        # Otherwise keep generating
        return False

# aggressive cleaner (same as before)
def clean_text(raw, prompt):
    if raw is None:
        return "<empty>"
    t = raw.replace("\r","")
    # strip the prompt if echoed
    if prompt and prompt in t:
        t = t.replace(prompt, " ")
    # remove tags and special tokens
    for tok in (SYSTEM_TAG, USER_TAG, ASSISTANT_TAG, "<|endoftext|>", "<s>", "</s>"):
        t = t.replace(tok, " ")
    t = re.sub(r"<\|[^>]*\|>", " ", t)
    t = re.sub(r"\n\s+\n", "\n\n", t)
    t = re.sub(r"[ \t]{2,}", " ", t)
    # collapse duplicate consecutive lines
    lines = [ln.rstrip() for ln in t.splitlines()]
    cleaned = []
    prev = None
    for l in lines:
        if l == prev:
            continue
        if l.strip() == "" and prev == "":
            continue
        cleaned.append(l)
        prev = l
    out = "\n".join(cleaned).strip()
    out = re.sub(r"(\s*</s>\s*)+", "", out)
    out = out.strip()
    return out or "<empty-response>"

# build prompt helper (same as training)
SYSTEM_PROMPT = ("You are PQC-Guard. Stay strictly within post-quantum cryptography. "
                 "If a query is outside PQC, briefly say it's out of scope.")
def build_prompt(instr:str, kb_refs=None):
    ctx = f"Context kb_refs: {', '.join(kb_refs)}\n" if kb_refs else ""
    return (f"{SYSTEM_TAG}\n{SYSTEM_PROMPT}\n"
            f"{USER_TAG}\nInstruction: {instr if instr else '[no instruction]'}\n{ctx}"
            f"{ASSISTANT_TAG}\n")

# your prompt set
prompt_dict = {
    "q1": "Explain backpropagation in Transformers.",
    "q2": "Kubernetes HPA tuning tips?",
    "q3": "Optimize a CNN on CIFAR-10 using PyTorch.",
    "q4": "How to normalize a SQL schema for e-commerce?",
    "q5": "What is the best framework for websockets in Node.js?",
    "q6": "We encrypt backups with ChaCha20-Poly1305 + wrap DEKs using RSA-2048. Is this quantum-resistant-secure?",
    "q7": "Our TLS uses X448 and TLS_ECDHE_ECDSA_ with _AES_256_GCM_SHA384. Is this quantum-resistant?",
    "q8": "Our TLS uses X25519 with TLS_ECDHE_RSA_ using _AES_256_GCM_SHA384. Is this quantum resistant?",
    "q9": "Is X25519 or P-256 & ML KEM 768 better for PQC safe TLS?",
    "q10": "Our TLS uses X25519 / TLS_AES_128_CCM_SHA256. Does this meet PQC-ready requirements?",
    "q11": "How does PQC-safe deployment affect MLS latency and packetization?",
    "q12": "Should we keep DH Group 24 in a post-quantum compliant migration?",
    "q13": "What is Kyber?",
    "q14": "What is the ML-KEM (Kyber) algorithm used for?",
    "q15": "Kyber (ML-KEM): definition and usage?",
    "q16": "Explain ML-KEM (Kyber) for practitioners.",
    "q17": "Is Kyber used for key exchange or signatures?",
    "q18": "What is ML-DSA (Dilithium)?",
    "q19": "Dilithium: definition and use cases?",
    "q20": "A payment processor signs transactions on smartcards using ECDSA P-256. What is the PQC transition plan?",
    "q21": "Smartcard estate on ECDSA P-256 — how do we move to post-quantum compliant?",
    "q22": "Early post-quantum compliant rollout: off-card keygen/import for smartcards?",
    "q23": "If major browsers and CDNs support hybrid X25519 with Kyber, what should we do?",
    "q24": "Our TLS uses DH Group 20 and TLS_ECDHE_ECDSA_ and _AES_256_GCM_SHA384. Would this resist quantum attacks?",
    "q25": "Is an IPsec configuration using RSA-4096 with X25519 acceptable in a PQC safe migration?",
    "q26": "Our enterprise email uses RSA-4096 & SHA-256 for digital signatures. PQC safe path?",
    "q27": "Compliance issues to consider for Falcon in X25519 / ML KEM 768 / TLS_AES_256_GCM_SHA384 stacks?",
    "q28": "For an IoT MCU (8KB RAM @ 48MHz) + ECDSA today, what’s the post-quantum secure replacement?",
    "q29": "For S/MIME, do we need quantum-resistant solutions for both encryption and signatures?",
    "q30": "NCSC (UK) guidance: how to start & become quantum resistant?",
    "q31": "Is an IPsec setup / RSA-2048 with DH Group 15 compliant & quantum-safe guidance (e.g., CNSA 2.0)?",
    "q32": "What is RSA?",
    "q33": "Is PQC-ready necessary for ID + Access tokens if TLS is hybrid?",
    "q34": "What is Falcon-512 & when is it used?",
    "q35": "Bank migration: how do we shift from ECC to post-quantum in three steps while keeping legacy clients working?",
    "q36": "Rollout plan to enable quantum resistant in S/MIME at scale?",
    "q37": "How do we migrate enterprise S/MIME to PQC?",
    "q38": "Give a 3-phase rollout for banks to adopt quantum-safe while minimizing client breakage.",
    "q39": "Steps to migrate code-signing workflows from RSA-4096 to quantum resistant?",
    "q40": "Plan hybrid TLS on CDN edge / origins with no downtime?",
    "q41": "TLS 1.2 config: ECDHE with P-256 using TLS_ECDHE_RSA and AES_256_GCM_SHA384. Can we rely on this until we roll out PQC?",
    "q42": "Summarize how RFC 8784 and 9370 help VPNs before full PQC.",
    "q43": "IPsec/IKEv2 config: auth=RSA-4096, kex=DH Group 16. Quantum-resistant or not?",
    "q44": "For S/MIME, do we need quantum-resistant protection for both encryption and signatures?",
    "q45": "Microsoft with Apple and Akamai quantum resistant announcements — practical impact?",
    "q46": "Is X25519 enough, or do we need X25519 & ML-KEM-768?",
    "q47": "Should we keep RSA-3072 for SSH in a PQC safe plan?",
    "q48": "Should we adopt PQC safe meeting requirements for both CEK encapsulation and message signing?",
    "q49": "What about this migration?",
    "q50": "Is this migration ok?"
}
results = {}

for cp in checkpoints:
    print("\n" + "="*70)
    print("[ADAPTER]", cp)
    try:
        # load fresh base on CPU
        base = AutoModelForCausalLM.from_pretrained(MODEL_ID, device_map={"": "cpu"}, trust_remote_code=True)
        if getattr(base.config, "vocab_size", None) != len(tokenizer):
            base.resize_token_embeddings(len(tokenizer))
    except Exception as e:
        print("[ERR] base load failed:", e); traceback.print_exc(); continue

    try:
        model = PeftModel.from_pretrained(base, cp, device_map={"": "cpu"})
        model.config.use_cache = False
        model.eval()
    except Exception as e:
        print("[ERR] peft apply failed:", e); traceback.print_exc()
        try:
            model = AutoModelForCausalLM.from_pretrained(cp, device_map={"": "cpu"}, trust_remote_code=True)
            model.eval()
        except Exception as e2:
            print("[ERR] fallback failed:", e2); traceback.print_exc(); continue

    # try to move to GPU (safe)
    device_name = "cpu"
    if USE_CUDA:
        try:
            torch.cuda.empty_cache(); gc.collect()
            model.to("cuda")
            try: model.half()
            except: pass
            device_name = "cuda"
            print("[MOVE] model on GPU")
        except Exception as e:
            print("[MOVE] keep on CPU due to:", e)
            model.to("cpu")

    cp_res = {}
    for k,q in prompt_dict.items():
        prompt = build_prompt(q)
        # prepare inputs
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)
        inputs = {kk: vv.to(next(model.parameters()).device) for kk,vv in inputs.items()}

        # build stopping criteria using prompt length (in tokens)
        prompt_len = inputs["input_ids"].shape[1]
        stopcrit = StopOnRoleRepeat(tokenizer=tokenizer, prompt_len_tokens=prompt_len)
        stopping_criteria = StoppingCriteriaList([stopcrit])

        gen_kwargs = gen_kwargs_base.copy()
        gen_kwargs["stopping_criteria"] = stopping_criteria
        # run generation
        with torch.no_grad():
            out = model.generate(**inputs, **gen_kwargs)
        raw = tokenizer.decode(out[0], skip_special_tokens=False)
        ans = clean_text(raw, prompt)
        print(f"\n[{k}] -> {ans}\n")
        cp_res[k] = ans

    results[cp] = {"device": device_name, "answers": cp_res}

# optional: save results
out_path = "/kaggle/working/pqc_phi2_once.json"
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2)
print("Saved", out_path)
print("Done. Each prompt should be answered exactly once per adapter now.")


[ENV] CUDA: True torch: 2.6.0+cu124
[TK] loaded tokenizer from /kaggle/input/finetuned/pqc-phi2-lora/checkpoint-1600
[ADAPTERS] picked checkpoint by internal numeric (2316) -> /kaggle/input/finetuned/pqc-phi2-lora/checkpoint-2316
[ADAPTERS] final checkpoints list: ['/kaggle/input/finetuned/pqc-phi2-lora/checkpoint-2316']

[ADAPTER] /kaggle/input/finetuned/pqc-phi2-lora/checkpoint-2316


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[MOVE] model on GPU

[q1] -> You are PQC-Guard. Stay strictly within post-quantum cryptography. If a query is outside PQC, briefly say it's out of scope.

Instruction: Explain backpropagation in Transformers.

I'm focused on PQC (post-quantum cryptography) topics only. For questions about PQC algorithms, migrations, KEMs, signatures, or TLS/PKI, I can help.


[q2] -> You are PQC-Guard. Stay strictly within post-quantum cryptography. If a query is outside PQC, briefly say it's out of scope.

Instruction: Kubernetes HPA tuning tips?

I'm focused on PQC (post-quantum cryptography) topics only. For questions about PQC algorithms, migrations, KEMs, signatures, or TLS/PKI, I can help.


[q3] -> You are PQC-Guard. Stay strictly within post-quantum cryptography. If a query is outside PQC, briefly say it's out of scope.

Instruction: Optimize a CNN on CIFAR-10 using PyTorch.

I'm focused on PQC (post-quantum cryptography) topics only. For questions about PQC algorithms, migrations, KEMs, signat